## 📝 Config & Imports
- 1024 max length has been working well for me. As some samples are longer, you may want to go as high as you can 

In [1]:
!pwd
!mkdir my-model
!cp /kaggle/input/my-model/__notebook__.ipynb /kaggle/working/my-model/__notebook__.ipynb
!chmod 777 /kaggle/working/my-model/__notebook__.ipynb
!jupyter nbconvert --to script /kaggle/working/my-model/__notebook__.ipynb
!cp /kaggle/working/my-model/__notebook__.py /kaggle/working/my_model.py

/kaggle/working
[NbConvertApp] WARNING | Config option `kernel_spec_manager_class` not recognized by `NbConvertApp`.
[NbConvertApp] Converting notebook /kaggle/working/my-model/__notebook__.ipynb to script
[NbConvertApp] Writing 7931 bytes to /kaggle/working/my-model/__notebook__.py


In [2]:
!pwd
!mkdir my-dataset
!cp /kaggle/input/my-dataset/__notebook__.ipynb /kaggle/working/my-dataset/__notebook__.ipynb
!chmod 777 /kaggle/working/my-dataset/__notebook__.ipynb
!jupyter nbconvert --to script /kaggle/working/my-dataset/__notebook__.ipynb
!cp /kaggle/working/my-dataset/__notebook__.py /kaggle/working/my_dataset.py

/kaggle/working
[NbConvertApp] WARNING | Config option `kernel_spec_manager_class` not recognized by `NbConvertApp`.
[NbConvertApp] Converting notebook /kaggle/working/my-dataset/__notebook__.ipynb to script
[NbConvertApp] Writing 5642 bytes to /kaggle/working/my-dataset/__notebook__.py


In [3]:
!pip install seqeval evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=04df4e05d69f35c868fc8528e2d0c2bbfb1aa99d5383aa99d0fb32d74e066eab
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [4]:
TRAINING_MODEL_PATH = "microsoft/deberta-v3-base"
TRAINING_MAX_LENGTH = 1024
OUTPUT_DIR = "output"
STRIDE=384

In [5]:
import os
import json
import argparse
from itertools import chain
from functools import partial

import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification
from datasets import Dataset, features
import numpy as np
from pathlib import Path
import evaluate
from enum import Enum
import sys
import random

import my_model
import my_dataset

2024-05-04 16:08:05.700420: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 16:08:05.700525: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 16:08:05.820727: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## 🗺️ Data Selection and Label Mapping
- As mentioned before, I additionaly use the moth dataset

In [6]:
base_input = Path("/kaggle/input/pii-detection-removal-from-educational-data")
data = json.load((base_input / "train.json").open("r"))
nicholas_data = json.load(open("/kaggle/input/pii-dd-mistral-generated/mixtral-8x7b-v1.json"))

In [7]:
all_labels = sorted(list(set(chain(*[x["labels"] for x in data]))))
label2id = {l: i for i,l in enumerate(all_labels)}
id2label = {v:k for k,v in label2id.items()}

print(id2label)

{0: 'B-EMAIL', 1: 'B-ID_NUM', 2: 'B-NAME_STUDENT', 3: 'B-PHONE_NUM', 4: 'B-STREET_ADDRESS', 5: 'B-URL_PERSONAL', 6: 'B-USERNAME', 7: 'I-ID_NUM', 8: 'I-NAME_STUDENT', 9: 'I-PHONE_NUM', 10: 'I-STREET_ADDRESS', 11: 'I-URL_PERSONAL', 12: 'O'}


## ♟️ Dataset (from My Dataset notebook)


In [8]:
ds = my_dataset.TokenizedDataset(data, nicholas_data, label2id, TRAINING_MAX_LENGTH, paragraph_augmentation=True, oversampling=True)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Oversampling essays with PIIs


## Model (from My Dataset notebook)

In [9]:
tokenizer = ds.tokenizer
collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)
model = my_model.from_pretrained(
    TRAINING_MODEL_PATH,
    num_labels=len(all_labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
) 

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForTokenClassificationPII were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 🧮 Competition metrics
- Note that we are not using the normal F1 score.
- Although it is early in the competition, there are plenty of discsussions already explaining this:
- e.g., here: https://www.kaggle.com/competitions/pii-detection-removal-from-educational-data/discussion/470024

In [10]:
from seqeval.metrics import recall_score, precision_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

def compute_metrics(p, all_labels):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [all_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [all_labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    recall = recall_score(true_labels, true_predictions)
    precision = precision_score(true_labels, true_predictions)
    f1_score = (1 + 5*5) * recall * precision / (5*5*precision + recall)
    
    results = {
        'recall': recall,
        'precision': precision,
        'f1': f1_score
    }
    return results

## 🏋🏻‍♀️ Training
- I actually do not use an eval set for submission to train on all data
- Values are not really tuned and go by gut feeling, as this is my first iteration / baseline

In [11]:
# I actually chose to not use any validation set. This is only for the model I use for submission.
args = TrainingArguments(
    output_dir=OUTPUT_DIR, 
    fp16=True,
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    report_to="none",
    evaluation_strategy="no",
    do_eval=False,
    save_total_limit=1,
    logging_steps=20,
    lr_scheduler_type='cosine',
    metric_for_best_model="f1",
    greater_is_better=True,
    warmup_ratio=0.1,
    weight_decay=0.01
)

trainer = Trainer(
    model=model, 
    args=args, 
    train_dataset=ds,
    data_collator=collator, 
    tokenizer=tokenizer,
    compute_metrics=partial(compute_metrics, all_labels=all_labels),
)

In [12]:
# %%time
trainer.train()

Step,Training Loss
20,2.844300
40,2.761400
60,2.659800
80,2.436600
100,2.209700
120,1.836100
140,1.443400
160,1.240100
180,0.927700
200,0.670600


TrainOutput(global_step=4692, training_loss=0.09830727315137047, metrics={'train_runtime': 5726.149, 'train_samples_per_second': 6.554, 'train_steps_per_second': 0.819, 'total_flos': 1.4060789391201024e+16, 'train_loss': 0.09830727315137047, 'epoch': 3.0})

## 💾 Save models
- You can click on "Save version" (top right) and "Save & Run All (Commit)"
- Then you can use this notebook as input for your inference notebook

In [13]:
trainer.save_model("deberta3base_1024_with_paragraphs_augmentation_weighted_loss_with_nicholas_new_hyper")
tokenizer.save_pretrained("deberta3base_1024_with_paragraphs_augmentation_weighted_loss_with_nicholas_new_hyper")

('deberta3base_1024_with_paragraphs_augmentation_weighted_loss_with_nicholas_new_hyper/tokenizer_config.json',
 'deberta3base_1024_with_paragraphs_augmentation_weighted_loss_with_nicholas_new_hyper/special_tokens_map.json',
 'deberta3base_1024_with_paragraphs_augmentation_weighted_loss_with_nicholas_new_hyper/spm.model',
 'deberta3base_1024_with_paragraphs_augmentation_weighted_loss_with_nicholas_new_hyper/added_tokens.json',
 'deberta3base_1024_with_paragraphs_augmentation_weighted_loss_with_nicholas_new_hyper/tokenizer.json')